In [36]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score
# 提取和组合值到向量用于 X

with open('updated_before_random_forest.pkl', 'rb') as f:
    df_list = pickle.load(f)

sensitive_dict_imm_1_weighted = {'matter': -1, 'pedophiles': -1, 'pedophile': -1, 'go': -1, 'see': -1, 'date': -1, 'stop': -1, 'attracted': -1, 'gay': -1, 'happened': -1, 'know': -1, 'risk': -1, 'way': -2, 'speech': -3, 'still': -3, 'penis': -3, 'said': -3, 'cum': -3, 'law': -4, 'rape': -5, 'sexual': -6, 'trans': -7}

In [37]:
import spacy
from collections import Counter

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# def calculate_word_frequencies(text):
#     # Tokenize the text
#     doc = nlp(text)
#     
#     # Extract tokens
#     tokens = [token.text for token in doc]
#     
#     # Count occurrences of each word
#     word_counts = Counter(tokens)
#     
#     # Calculate total number of tokens
#     total_tokens = len(tokens)
#     
#     # Calculate word frequencies
#     word_frequencies = {word: count / total_tokens for word, count in word_counts.items()}
#     
#     return word_frequencies
# 
# # Example usage
# text = "This is a sample text. This text is for testing."
# frequencies = calculate_word_frequencies(text)
# print(frequencies)

In [38]:
def calculate_weighted_score(text, sensitive_dict):
    # Tokenize the text
    doc = nlp(text)
    
    # Extract tokens
    tokens = [token.text for token in doc]
    
    # Initialize score
    score = 0
    
    # Calculate score
    for token in tokens:
        if token in sensitive_dict:
            score += sensitive_dict[token]
            
    score/=len(tokens)
    
    return score

In [39]:
for df in df_list:
    for index, row in df.iterrows():
        text = row['text']
        df.at[index, 'sensi_score'] = calculate_weighted_score(text, sensitive_dict_imm_1_weighted)

In [41]:
vectors = []
targets = []

for df in df_list:
    for index, row in df.iterrows():
        # print(1)
        vector = row['txt_sentiment'] + row['txt_blob'] + [row['conv_pos']] + [row['token_count']] + [row['meta.score']] + [row['sensi_score']]
        vectors.append(vector)
        
        x = (int(row['imm_check']) + int(row['imm_1_check']))
        if x==0:
            a=1
        else:
            a=0
        targets.append(a)

X = np.array(vectors)

Y = np.array(targets)

rf = RandomForestClassifier(random_state=42)

# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Perform cross-validation
cv_results = cross_validate(rf, X, Y, cv=5, scoring=scoring)

rf.fit(X, Y)

# 获取特征重要性
feature_importances = rf.feature_importances_


column_labels = [
    'pos', 'neg', 'neu', 'compound',
    'polarity', 'subjectivity',
    'conv_pos', 'token_count', 'score', 'sensi_score'
]

# 打印特征重要性
feature_importance_df = pd.DataFrame({
    'feature': column_labels,
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

print(feature_importance_df)

# Print evaluation metrics
for metric in scoring.keys():
    print(f"{metric}: {np.mean(cv_results['test_' + metric])}")

C:\Users\L\anaconda3\envs\Raja\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\L\anaconda3\envs\Raja\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\L\anaconda3\envs\Raja\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\L\anaconda3\envs\Raja\Lib\site-packag

        feature  importance
9   sensi_score    0.117288
7   token_count    0.109433
4      polarity    0.108776
3      compound    0.106252
2           neu    0.105821
5  subjectivity    0.105519
1           neg    0.097987
6      conv_pos    0.097410
0           pos    0.084158
8         score    0.067355
accuracy: 0.9520249418024609
precision: 0.0
recall: 0.0
f1: 0.0
roc_auc: 0.5


In [13]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle

# 定义随机森林的超参数网格
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 创建随机森林分类器
rf = RandomForestClassifier(random_state=42)

# 使用 GridSearchCV 进行超参数搜索
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# 进行超参数搜索
grid_search.fit(X, Y)
loaded_grid_search = grid_search
# 保存计算结果到文件
# with open('grid_search_results.pkl', 'wb') as f:
#     pickle.dump(grid_search, f)
# 
# # 加载计算结果
# with open('grid_search_results.pkl', 'rb') as f:
#     loaded_grid_search = pickle.load(f)

# 打印最佳结果
print(f"Best parameters found: {loaded_grid_search.best_params_}")
print(f"Best cross-validation score: {loaded_grid_search.best_score_}")


scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, average='macro', multi_class='ovr')
}

def auto_search(X, Y, grid=param_grid):
    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X, Y)
    
    # 提取所有评分指标的结果
    cv_results = grid_search.cv_results_
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    # 提取最佳参数的所有评分指标
    best_index = grid_search.best_index_
    best_scores = {metric: cv_results[f'mean_test_{metric}'][best_index] for metric in scoring.keys() if f'mean_test_{metric}' in cv_results}
    return grid_search, grid_search.best_params_, grid_search.best_score_,best_scores

Best parameters found: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best cross-validation score: 0.95228300631859


In [24]:
import pandas as pd
from tabulate import tabulate

def print_dataframe(df):
    print(tabulate(df, headers='keys', tablefmt='psql'))

In [35]:
combined_df = pd.concat(df_list, ignore_index=True)
# print_dataframe(combined_df[['text', 'imm_check', 'imm_1_check', 'sensi_score'][0:1]])
print(combined_df[['text','imm_check', 'imm_1_check', 'sensi_score']].head(5).values)

[['&gt;So within our predefined two genders, we have a huge range of variability. Is a "tomboy" type woman "crazy" for thinking that the hyper feminine stereotype of women is stupid, and that she should be able to do "manly" things without being called butch?\n\n\nI never said anything like that. In fact I\'ve said the opposite. If someone wants to be a tomboy, whatever, go for it. That\'s individualism. But saying you\'re a boy because you act manly according to stereotypes when you\'re factually female is the same as saying you\'re a different race because you fit racial stereotypes regardless of the fact that you have non of that racial blood. It\'s gross. '
  True True -0.050359712230215826]
 ["&gt; But saying you're a boy because you act manly according to stereotypes when you're factually female is the same as saying you're a different race because you fit racial stereotypes regardless of the fact that you have non of that racial blood. It's gross.\n\nSaying you're male *has noth

In [ ]:
t = "Because I have a penis, and balls, and my chromosomes. I can cum. I don't have any extra packages."

